In [90]:
import pandas as pd
import numpy as np
import pickle
import math
import warnings
# !pip install yfinance
import yfinance as yf
"""Evaluate trading strategy performance over time interval."""
from typing import List, Tuple, Dict
from datetime import datetime
#import datetime

warnings.filterwarnings("ignore")



In [91]:
#train_df = pd.read_csv('8k_data_labels.csv')

In [92]:
def load_historical_data(filename):
    with open(filename, 'rb') as handle:
        tikr_dict = pickle.load(handle)
        return tikr_dict
TIKRS_dat = load_historical_data('TIKR_DATA.pickle')

In [175]:
class StockSimulation:
    def __init__(self, cash=1000000, tikrs = ['aapl','msft'], historical_data=TIKRS_dat):
        self.cash = cash
        self.tikrs = tikrs
        self.portfolio = {}
        for tikr in tikrs:
            self.portfolio[tikr] = 0
        self.active_log = []
        self.transaction_log = []
        self.transaction_cost = 0.01
        self.historical_data = TIKRS_dat
        
    def get_price(self, tikr, date):
        start = date
        if type(start) is str:
            start = datetime.strptime(start, '%Y%m%d')
          
        date = start.strftime('%Y-%m-%d')

        company_df = self.historical_data[tikr]
        return company_df[company_df['Date'] > date].iloc[0]['Open']
#         return yf.download(
#             tikr, 
#             start=start,
#             interval= '1d', 
#             progress= False, ).loc[date]['Adj Close']



    def buy(self, tikr, date, allocated_money):
        # check if there is enough cash to buy
        if self.cash < allocated_money:
            print("Not enough cash to buy")
            return

        # calculate number of shares to buy
        price = self.get_price(tikr, date)
        shares = allocated_money / price

        # update portfolio and cash balance
        if tikr in self.active_log:
            self.portfolio[tikr] += shares
        else:
            self.portfolio[tikr] = shares
            self.active_log.append(tikr)
        # minus the cost of stock
        stock_cost = shares * price
        # minus trasaction cost
        #self.cash -= stock_cost * (1 + self.transaction_cost)
        self.cash -= stock_cost 


        # log transaction
        self.transaction_log.append({
            "type": "buy",
            "tikr": tikr,
            "date": date,
            "shares": shares,
            "price": price,
            "amount": stock_cost ,
            "transaction_cost": 0,
            "balance": self.active_balance(date)
        })

    def sell(self, tikr, date, allocated_money):
        # check if there are enough shares to sell
        if tikr not in self.portfolio:
            print("No shares of {} in portfolio".format(tikr))
            return


        # calculate number of shares to sell
        price = self.get_price(tikr, date)
        shares = allocated_money / price


        if self.portfolio[tikr] < shares:
            print("Not enough shares of {} to sell".format(tikr))
            return

        # update portfolio and cash balance
        self.portfolio[tikr] -= shares
        if self.portfolio[tikr] == 0:
            self.active_log.remove(tikr)

        # plus the earning
        stock_cost = shares * price
        self.cash += stock_cost
        # minus trasaction cost
        self.cash -= stock_cost *  self.transaction_cost


        # log transaction
        self.transaction_log.append({
            "type": "sell",
            "tikr": tikr,
            "date": date,
            "shares": shares,
            "price": price,
            "amount": stock_cost ,
            "transaction_cost": stock_cost * self.transaction_cost,
            "balance": self.active_balance(date)
        })

    #TODO
    def get_next_trading_date(self,tikr, date):
        return None

    def rebalance(self,percentage,date):
        buy = []
        
        
        true_balance = self.active_balance(date) * 0.99
        for tikr, percent in zip(self.tikrs, percentage):
            price = self.get_price(tikr, date)
            expected_value = true_balance * percent
            current_value = self.portfolio[tikr] * price
            allocated_money = expected_value - current_value

            if allocated_money < 0:
                self.sell(tikr, date, -1 * allocated_money)
                #total_transaction += self.transaction_log[-1]['transaction_cost']
            else:
                buy += [(tikr, allocated_money )]
        
       # avg_transaction = total_transaction/len(buy)
        for tikr, allocated_money in buy:
            self.buy(tikr, date, allocated_money )#- avg_transaction)




    def active_balance(self, date):
        balance = self.cash
        for tikr in self.active_log:
            price = self.get_price(tikr, date)
            balance += self.portfolio[tikr] * price
        return balance
    
    def print_portfolio(self, date):
        balance = self.active_balance(date)
        for tikr in self.tikrs:
            price = self.get_price(tikr, date)
            tikr_holding = self.portfolio[tikr] * price
            print(tikr, tikr_holding/balance )
            

    def transaction_summary(self):
        for txn in self.transaction_log:
            print("{} {} {} shares of {} at ${:.2f} for ${:.2f} (transaction cost: ${:.2f}), balance: ${:.2f}".format(
                txn['date'], txn['type'], txn['shares'], txn['tikr'], txn['price'], txn['amount'], txn['transaction_cost'], txn['balance']))


In [176]:
a =[[1,2,3],[3,2,1]]
np.argmax(a, axis=1)

array([2, 0])

In [177]:



""" All trading strategies should have standardized input to work in automated
    downstream testing."""

"""
def trading_strategy(
        dates,
        predictions,
        company_list: List[str]
            ) -> Dict[str, List[float]]:
    
    strategy = dict()
    predictions = np.argmax(predictions, axis=1)
    for date, tikr, pred in zip(dates, tikrs, predictions):
        if pred == 0: #underperforms
            
            #sell
            
            
        
    
    return strategy
"""

def get_strategy_annual_return(
        strategy: Dict[str, List[float]],
        company_list: List[str],
        end_date,
        starting_balance: int = 1e7,
        start_date: datetime = '20000101') -> float:
    """
    Calculate annual return over time period.
    
    Parameters
    ----------
    strategy: Dict[datetime, List[float]]
        A dictionary of portfolio rebalance dates associated with portfolio
        allocation percentages. The portfolio allocation percentages share the
        same indexing as company_list.
    company_list: List[str]
        The companies invested into by the trading strategy percent allocation.
    start_date: datetime, str
        Accepted format "year_month_day", or datetime object. The first day of
        trading.
    end_date: datetime, str
        Accepted format "year_month_day", or datetime object. The final day of
        trading used to determine net worth and annualized return.
    starting_balance: int
        The initial amount of money invested.
    Returns
    -------
    Annualized Return: float
        The annualized return rate, where 0% return indicates 1.00.
    """

    if type(start_date) is str:
        start_date = datetime.strptime(start_date, "%Y%m%d")
    if type(end_date) is str:
        end_date = datetime.strptime(end_date, "%Y%m%d")

    s = StockSimulation(cash = starting_balance, tikrs = company_list)
    for date, portfolio_allocation in strategy.items():
        try:
            s.rebalance(percentage= portfolio_allocation, date = date)
            print(date, " balance:",s.active_balance(date))
        except:
            print('rebalance')




    # At each date, rebalance current networth to be distributed
    # percentage-wise between companies in portfolio_allocations

    return s.active_balance(end_date)/starting_balance

In [178]:
s = StockSimulation()
s.get_price('msft', '20000110')
s.buy('msft', '20000110', 10000)
s.sell('msft', '20000110', 10000)
s.active_balance('20000110')


999900.0

In [179]:
s = StockSimulation(cash =1000)
s.rebalance([0.5,0.5],'20001210')
s.print_portfolio( '20001210')
s.rebalance([0.3,0.7], "20011210")
s.print_portfolio( "20011210")
s.rebalance([0,0],'20100112')

aapl 0.495
msft 0.495
aapl 0.29774248620736027
msft 0.694732467817174


In [173]:
s.transaction_summary()

20001210 buy 1825.1875346835704 shares of aapl at $0.27 for $495.00 (transaction cost: $0.00), balance: $1000.00
20001210 buy 17.83783783783784 shares of msft at $27.75 for $495.00 (transaction cost: $0.00), balance: $1000.00


8132.000126617264

In [161]:
company_list = ['aapl', 'msft']
end_date = "20100112"
start_date = '20001210'
starting_balance = 1000
strategy = {"20001210": [0.5,0.5], "20011210": [0.3,0.7],"20100112": [0,0] }

In [162]:
get_strategy_annual_return(strategy,company_list,end_date,starting_balance, start_date)

20001210  balance: 1000.0
20011210  balance: 1348.9560177068931
20100112  balance: 8132.000126617264


8.132000126617264